In [1]:
%load_ext autoreload
%autoreload 2
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('klue/roberta-large')

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at klue/roberta-large were not used when initializing BertModel: ['roberta.encoder.layer.17.attention.output.LayerNorm.weight', 'roberta.encoder.layer.13.attention.output.dense.weight', 'roberta.encoder.layer.17.output.LayerNorm.bias', 'roberta.encoder.layer.18.attention.output.dense.weight', 'lm_head.layer_norm.bias', 'roberta.encoder.layer.1.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.attention.self.query.bias', 'roberta.encoder.layer.7.output.LayerNorm.weight', 'roberta.encoder.layer.8.output.dense.weight', 'roberta.encoder.layer.14.intermedia

In [11]:
from Fishing.dataset import BaitDataset
config = {
    'max_word_len': 128,
    'bait_path': '/workspace/codes/02_DSBA_Project/bait_news_gen/joonghoon/t5_base/tfidf_content/content_chunking_forward',
    'data_path': '/workspace/codes/02_DSBA_Project/bait_news_gen/data/original',
    'sort': 'News_Direct',
}
dataset = BaitDataset(config, split='train', tokenizer=tokenizer)
bait_title, news_title, bait_file_path = dataset.load_bait_news_info(data_dir=config['data_path'], bait_dir=config['bait_path'], split='train')

Load Data train | News : 100%|██████████| 39996/39996 [00:02<00:00, 14516.25it/s]
Load Data train | Bait : 100%|██████████| 36969/36969 [00:02<00:00, 13155.77it/s]
Load Data train | Bait : 100%|██████████| 39996/39996 [00:03<00:00, 12855.55it/s]
Load Data validation | Bait : 100%|██████████| 39996/39996 [00:03<00:00, 13104.79it/s]
Load Data test | Bait : 100%|██████████| 39996/39996 [00:03<00:00, 12823.02it/s]


In [12]:
len(bait_title), len(news_title)

(39996, 39996)

In [17]:
refs = list(news_title.values())
cands = list(bait_title.values())
from KoBERTScore.KoBERTScore import BERTScore
kobertscore = BERTScore(model_name_or_path='klue/roberta-large')
score = kobertscore(refs, cands)

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at klue/roberta-large were not used when initializing BertModel: ['roberta.encoder.layer.17.attention.output.LayerNorm.weight', 'roberta.encoder.layer.13.attention.output.dense.weight', 'roberta.encoder.layer.17.output.LayerNorm.bias', 'roberta.encoder.layer.18.attention.output.dense.weight', 'lm_head.layer_norm.bias', 'roberta.encoder.layer.1.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.attention.self.query.bias', 'roberta.encoder.layer.7.output.LayerNorm.weight', 'roberta.encoder.layer.8.output.dense.weight', 'roberta.encoder.layer.14.intermediate.dense.weight', 'roberta.encoder.layer.8.attention.self.key.bias', 'roberta.encoder.layer.12.attention.output.dense.bias', 'roberta.encoder.layer.3.intermediate.dense.bias', 'roberta.encoder.layer.7.attention.self.key.bias', 'roberta.encoder.

Load klue/roberta-large with 22 layers


Calculating BERTScore: 100%|██████████| 313/313 [01:27<00:00,  3.57it/s]


In [26]:
import torch
index = torch.sort(torch.Tensor(score))[1]
score = torch.sort(torch.Tensor(score))[0]

In [52]:
index_ = index[-15000]
print(f"Score: {score[index_]}")
print(f"Reference: {refs[index_]}")
print(f"Candidate: {cands[index_]}")

Score: 0.9535970091819763
Reference: 삼성디스플레이, 최대 10조원 OLED 투자 '스타트'
Candidate: 삼성디스플레이, 올해 OLED 설비투자 10조원...2년 연속 뭉칫돈


In [1]:
import pandas as pd
from KoBERTScore.KoBERTScore import BERTScore
from transformers import AutoTokenizer

data_path = '../data/Fake/content_chunking_backward/generated/fake_top3.csv'

df = pd.read_csv(data_path)
refs = df['original_title'].tolist()
preds = df['bait_title'].tolist()

bert_scorer = BERTScore(model_name_or_path = 'klue/roberta-large')
score = bert_scorer.score(refs, preds)
df['BERTScore'] = score

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-24 04:42:03.874497: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-24 04:42:04.755925: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/python3.8/site-packages/torch/lib:/opt/conda/lib/python3.8/site-packages/torch_tensorrt/lib:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/

KeyboardInterrupt: 

In [4]:
import os
os.path.basename('../data/Fake/content_chunking_backward/generated/fake_top3.csv').split('.')[0]

'fake_top3'

In [2]:
import pandas as pd

data_path = '../data/Fake/content_rotation_backward/filtered/fake_top3_90_100.csv'
data_path = '../data/Fake/content_rotation_backward/generated/fake_top3.csv'

df = pd.read_csv(data_path)

In [3]:
df

,news_id,original_title,content,sim_news_id,bait_title,category,label,bait_content
0,PO_M03_417681,정경두 “SLBM 도발은 남북군사합의에 없다”,정경두 국방부 장관은 2일 국회 국방위원회 국정감사에서 북한의 이날 미사일 발사가 ...,PO_M03_108013,"정의용 \""北 미사일 발사, 9·19 군사합의 위반 없잖아\""",PO,0,정의용 청와대 국가안보실장이 6일 최근 잇따른 북한의 미사일 발사가 9·19 남북군...
1,IS_M10_079706,"상반기 사이버 보안 3대 위협, 랜섬웨어 그리고…",올해 상반기 국내외 사이버 보안 위협 특징으로 랜섬웨어가 꼽혔다.\nPC와 모바일에...,IS_M14_070355,"[이슈분석]북한 해킹 파문, \""국제 사이버 보안 위협\""",IS,0,한국을 노린 지능형지속위협(APT) 공격 징후가 포착됐다.\n북한으로 추정되는 해커...
2,EC_M05_227267,"대한항공-현대오일뱅크, 바이오항공유 협력 MOU",대한항공이 항공 부문 기후변화에 대응하기 위해 현대오일뱅크와 협력한다.\n대한항공과...,EC_M02_005162,"대한항공, 도심항공교통 환경 조성 위한 '바이오항공유' 조성",EC,0,"대한항공이 도심항공교통(UAM, Urban Air Mobility) 시대의 안전 운..."
3,SO_M06_436508,"성남시, 문화상 수상자 4명 선정…8일 시민의 날 시상","성남시는 ‘제29회 성남시 문화상 수상자’로 교육 부문 김학수(61), 예술 부문 ...",SO_M06_129262,"성남시, ‘성남박물관' 명칭 선정...제29회 성남시 문화상 선정",SO,0,성남시는 오는 2025년 상반기 개관을 목표로 수정구 신흥동에 건립 추진 중인 시립...
4,EC_M02_003013,"롯데하이마트, 조손가정 결연아동에 입학 격려 지원금 전달",롯데하이마트(대표 황영근)가 24일 입학을 기다리고 있는 조손가정 결연아동 33명을...,EC_M05_246826,"롯데하이마트, 해외 에너지 취약계층 아동, 친환경 태양광 랜턴 솔라미 전달",EC,0,롯데하이마트는 해외 에너지 소외계층 아동들을 위해 친환경 태양광 랜턴 솔라미를 전달...
...,...,...,...,...,...,...,...,...
131995,PO_M08_394777,"\""서울시장 가상 양자대결…오세훈 49.7% 송영길 36.9%\""",6·1 지방선거에서 국민의힘 서울시장 후보로 나서는 오세훈 현 서울시장이 더불어민주...,PO_M08_386027,"\""국민힘 김은혜· 김동연, 근소한 열세\""",PO,0,국민의힘 김은혜 경기지사 후보가 더불어민주당 김동연 경기지사 후보를 오차밤위 밖에서...
131996,SO_M07_155613,울산지검 '공무상 재해보상제도 안내서' 발간해 눈길,울산지방검찰청은 공무원 재해보상법 시행 후 처음으로 「공무상 재해보상제도 안내서」를...,SO_M02_126795,"전북, 중대재해 예방·체계 대응 계획 마련",SO,0,전북도는 오는 27일 '중대재해처벌 등에 관한 법률' 시행에 따라 중대재해 사전예방...
131997,GB_M11_060449,7m 악어 배 속에서 나온 30대 청년의 두 다리…주민들 '통곡',인도네시아령 파푸아의 한 마을에서 악어의 공격을 받고 실종된 30대 청년의 유해가 ...,GB_M11_055618,"中, \""한국서 참전군 유해 송환\""",GB,0,중국이 2일 한국으로부터 한국전쟁 참전군인 유해를 돌려받고 '영웅의 귀환'이라며 대...
131998,EC_M05_237797,"하나금융그룹, '하나 소셜벤처 아카데미' 3기 모집",하나금융그룹은 창업 교육을 통해 사회혁신 창업가를 육성하는 프로그램 '하나 소셜벤처...,EC_M04_015469,‘하나 소셜벤처 아카데미' 4개 대학 신규 선정,EC,0,중소기업청은 청년 창업 활성화와 창업 붐 확산을 위해 우수한 창업 인프라와 역량을 ...


In [6]:
132000-118424

13576

In [34]:
threshold_under = 0.99
threshold_upper = 1

len(df[threshold_under < df['BERTScore'] ][df['BERTScore'] < threshold_upper])

/tmp/ipykernel_30800/1348730651.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(df[threshold_under < df['BERTScore'] ][df['BERTScore'] < threshold_upper])


8

In [35]:
df[threshold_under < df['BERTScore'] ][df['BERTScore'] < threshold_upper][['original_title', 'bait_title', 'BERTScore']].head()

/tmp/ipykernel_30800/1622317083.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[threshold_under < df['BERTScore'] ][df['BERTScore'] < threshold_upper][['original_title', 'bait_title', 'BERTScore']].head()


,original_title,bait_title,BERTScore
19907,"요기요, '다회용기' 카테고리 신설","요기요, '다회용기' 카테고리 신설",0.998862
19987,"무신사, '랩비트 2022' 메인 스폰서 참여","무신사, ‘랩비트 2022' 메인 스폰서 참여",0.992801
20275,"‘신사와 아가씨’ 이세희, ‘귀여운 이소룡’ 변신","‘신사와 아가씨’ 이세희, '귀여운 이소룡’ 변신",0.993656
23703,"우주소녀 설아, 서울 억새 온라인 홍보 영상 출연","우주소녀 설아, 서울 억새 온라인 홍보 영상 출연",0.998313
42006,"광동제약, '귀한삼 홍삼녹용 발효원' 출시","광동제약, '귀한삼 홍삼녹용 발효원' 출시",0.991354


In [15]:
407*64


26048

In [3]:
import pandas as pd

# data_path = '../data/Fake/content_rotation_backward/filtered/fake_top3_90_100.csv'
# tfidf_data_path = '../data/Fake/tfidf/generated/fake_top3.csv'
data_path = '../data/Fake/tfidf/filtered/fake_top3_90_99.csv'
# data_path = '../data/Real/test.csv'
# data_path = '../data/Fake/fake_original.csv'

# tfidf_df = pd.read_csv(tfidf_data_path)
df = pd.read_csv(data_path)

In [4]:
df

,news_id,original_title,original_content,sim_news_id,fake_title,category,label,sim_news_content,sim_news_title,filter_bertscore
0,PO_M03_417681,정경두 “SLBM 도발은 남북군사합의에 없다”,정경두 국방부 장관은 2일 국회 국방위원회 국정감사에서 북한의 이날 미사일 발사가 ...,PO_M03_108013,"정의용 \""北 발사체 도발, 9·19 남북군사합의 위반 아니다\""",PO,0,정의용 청와대 국가안보실장이 6일 최근 잇따른 북한의 미사일 발사가 9·19 남북군...,"정의용 \""北 발사체 도발, 9·19 남북군사합의 위반 아니다\""",0.942056
1,IS_M10_079706,"상반기 사이버 보안 3대 위협, 랜섬웨어 그리고…",올해 상반기 국내외 사이버 보안 위협 특징으로 랜섬웨어가 꼽혔다.\nPC와 모바일에...,IS_M14_070355,한국 노린 APT공격 징후...10개 액티브X 취약점 이용,IS,0,한국을 노린 지능형지속위협(APT) 공격 징후가 포착됐다.\n북한으로 추정되는 해커...,한국 노린 APT공격 징후...10개 액티브X 취약점 이용,0.916675
2,EC_M05_227267,"대한항공-현대오일뱅크, 바이오항공유 협력 MOU",대한항공이 항공 부문 기후변화에 대응하기 위해 현대오일뱅크와 협력한다.\n대한항공과...,EC_M02_005162,"대한항공, 도심항공교통 시대 준비 나선다",EC,0,"대한항공이 도심항공교통(UAM, Urban Air Mobility) 시대의 안전 운...","대한항공, 도심항공교통 시대 준비 나선다",0.915871
3,SO_M06_436508,"성남시, 문화상 수상자 4명 선정…8일 시민의 날 시상","성남시는 ‘제29회 성남시 문화상 수상자’로 교육 부문 김학수(61), 예술 부문 ...",SO_M06_129262,"성남시, 성남시립박물관 명칭 ‘성남역사박물관’으로 결정",SO,0,성남시는 오는 2025년 상반기 개관을 목표로 수정구 신흥동에 건립 추진 중인 시립...,"성남시, 성남시립박물관 명칭 ‘성남역사박물관’으로 결정",0.939672
4,EC_M02_003013,"롯데하이마트, 조손가정 결연아동에 입학 격려 지원금 전달",롯데하이마트(대표 황영근)가 24일 입학을 기다리고 있는 조손가정 결연아동 33명을...,EC_M05_246826,"롯데하이마트, 해외 결연 아동에 태양광 랜턴 기증",EC,0,롯데하이마트는 해외 에너지 소외계층 아동들을 위해 친환경 태양광 랜턴 솔라미를 전달...,"롯데하이마트, 해외 결연 아동에 태양광 랜턴 기증",0.943000
...,...,...,...,...,...,...,...,...,...,...
129769,PO_M08_394777,"\""서울시장 가상 양자대결…오세훈 49.7% 송영길 36.9%\""",6·1 지방선거에서 국민의힘 서울시장 후보로 나서는 오세훈 현 서울시장이 더불어민주...,PO_M08_386027,"김은혜 47.9% vs 김동연 39.4%… 경기도민, 국정안정론에 51.3% 응답",PO,0,국민의힘 김은혜 경기지사 후보가 더불어민주당 김동연 경기지사 후보를 오차밤위 밖에서...,"김은혜 47.9% vs 김동연 39.4%… 경기도민, 국정안정론에 51.3% 응답",0.936452
129770,SO_M07_155613,울산지검 '공무상 재해보상제도 안내서' 발간해 눈길,울산지방검찰청은 공무원 재해보상법 시행 후 처음으로 「공무상 재해보상제도 안내서」를...,SO_M02_126795,중대재해법 내일부터 시행...전북도 대응계획 마련,SO,0,전북도는 오는 27일 '중대재해처벌 등에 관한 법률' 시행에 따라 중대재해 사전예방...,중대재해법 내일부터 시행...전북도 대응계획 마련,0.930591
129771,GB_M11_060449,7m 악어 배 속에서 나온 30대 청년의 두 다리…주민들 '통곡',인도네시아령 파푸아의 한 마을에서 악어의 공격을 받고 실종된 30대 청년의 유해가 ...,GB_M11_055618,"\""영웅 돌아왔다\""…한국전 전몰자 유해송환 中 애국주의 강조",GB,0,중국이 2일 한국으로부터 한국전쟁 참전군인 유해를 돌려받고 '영웅의 귀환'이라며 대...,"\""영웅 돌아왔다\""…한국전 전몰자 유해송환 中 애국주의 강조",0.929864
129772,EC_M05_237797,"하나금융그룹, '하나 소셜벤처 아카데미' 3기 모집",하나금융그룹은 창업 교육을 통해 사회혁신 창업가를 육성하는 프로그램 '하나 소셜벤처...,EC_M04_015469,중기청 창업선도대학 6곳 신규선정,EC,0,중소기업청은 청년 창업 활성화와 창업 붐 확산을 위해 우수한 창업 인프라와 역량을 ...,중기청 창업선도대학 6곳 신규선정,0.910519


In [14]:
df.rename(columns={'bait_title': 'fake_title', 'content': 'original_content'}, inplace=True)
df.to_csv(data_path, index=False)

In [10]:
# df['sim_news_content'] = df['bait_content']
# df['sim_news_title'] = tfidf_df['sim_news_title']
# df.rename(columns={'bait_title': 'fake_title', 'content': 'original_content'}, inplace=True)
# df['sim_news_title'] = df['fake_title']
# df.drop('bait_content', axis=1, inplace=True)

In [7]:
df

,news_id,original_title,original_content,sim_news_id,fake_title,category,label
0,PO_M02_107666,"Jeffrey Hollender, 지속가능한 콘돔 개발",20년전 제프리 홀렌더(Jeffrey Hollender)는 아마존산 고무로 만든 콘...,NaN,NaN,PO,1
1,SO_M06_454436,"GC녹십자그룹, 의료재단 등 계열사 앞세워 해외판로 개척 본격화",녹십자(006280)그룹이 GC녹십자 산하 3개 계열사와 GC녹십자의료재단을 앞세워...,NaN,NaN,SO,1
2,LC_M09_360758,‘공간사랑’ 7080 춤꾼들…‘2014 춤판의 틀을 깨라’,"머리는 옛 춤을 잊었지만, 몸은 춤을 기억한다.\n한국현대무용 1세대 춤꾼 이정희(...",NaN,NaN,LC,1
3,SO_M02_124375,"[기획기사] 스마일게이트, 청년의 게임을 향한 희망을 키우다",글로벌 게임 회사인 스마일게이트는 희망스튜디오라는 재단을 설립하여 게임과 관련된 진...,NaN,NaN,SO,1
4,SO_M06_446495,"진중권 \""'김만배 녹취'는 쉰 떡밥…선거 앞두고 공작\""","부산저축은행 불법 대출 사건과 관련, \""박영수 변호사와 윤석열 당시 대검 중수부 ...",NaN,NaN,SO,1
...,...,...,...,...,...,...,...
39995,ET_M03_285158,"풍선껌 이동욱, 이것이 바로 '츤데레 남사친'…설렘 폭발","풍선껌 이동욱, 이것이 바로 '츤데레 남사친' 이다 배우 이동욱이 '츤데레 남사친'...",NaN,NaN,ET,1
39996,GB_M11_042745,[원유마감]WTI 1.8% 상승…사우디 아람코 CEO 낙관론,국제유가가 올랐다.\n사우디 아라비아의 석유공사 아람코가 수요 회복을 기대하는 낙관...,NaN,NaN,GB,1
39997,ET_M13_023119,"다비치, 바이브 20주년 ‘REVIBE’ 두 번째 음원 ‘사진을 보다가’ 발매",여성 듀오 다비치(DAVICHI)가 재해석한 보컬 듀오 바이브(VIBE)의 ‘사진을...,NaN,NaN,ET,1
39998,PO_M01_403091,광우병 재발에 조중동 분열증 보도,이명박 정부의 미국 광우병 대처를 놓고 '거짓말' 논란이 증폭되는 가운데 보수언론은...,NaN,NaN,PO,1


In [14]:
df.to_csv(data_path, index=False)

In [15]:
df

,news_id,original_title,original_content,sim_news_id,fake_title,category,label,sim_news_content,sim_news_title
0,PO_M03_417681,정경두 “SLBM 도발은 남북군사합의에 없다”,정경두 국방부 장관은 2일 국회 국방위원회 국정감사에서 북한의 이날 미사일 발사가 ...,PO_M03_108013,"정의용 \""北 미사일 발사, 9·19 군사합의 위반 없잖아\""",PO,0,정의용 청와대 국가안보실장이 6일 최근 잇따른 북한의 미사일 발사가 9·19 남북군...,"정의용 \""北 발사체 도발, 9·19 남북군사합의 위반 아니다\"""
1,IS_M10_079706,"상반기 사이버 보안 3대 위협, 랜섬웨어 그리고…",올해 상반기 국내외 사이버 보안 위협 특징으로 랜섬웨어가 꼽혔다.\nPC와 모바일에...,IS_M14_070355,"[이슈분석]북한 해킹 파문, \""국제 사이버 보안 위협\""",IS,0,한국을 노린 지능형지속위협(APT) 공격 징후가 포착됐다.\n북한으로 추정되는 해커...,한국 노린 APT공격 징후...10개 액티브X 취약점 이용
2,EC_M05_227267,"대한항공-현대오일뱅크, 바이오항공유 협력 MOU",대한항공이 항공 부문 기후변화에 대응하기 위해 현대오일뱅크와 협력한다.\n대한항공과...,EC_M02_005162,"대한항공, 도심항공교통 환경 조성 위한 '바이오항공유' 조성",EC,0,"대한항공이 도심항공교통(UAM, Urban Air Mobility) 시대의 안전 운...","대한항공, 도심항공교통 시대 준비 나선다"
3,SO_M06_436508,"성남시, 문화상 수상자 4명 선정…8일 시민의 날 시상","성남시는 ‘제29회 성남시 문화상 수상자’로 교육 부문 김학수(61), 예술 부문 ...",SO_M06_129262,"성남시, ‘성남박물관' 명칭 선정...제29회 성남시 문화상 선정",SO,0,성남시는 오는 2025년 상반기 개관을 목표로 수정구 신흥동에 건립 추진 중인 시립...,"성남시, 성남시립박물관 명칭 ‘성남역사박물관’으로 결정"
4,EC_M02_003013,"롯데하이마트, 조손가정 결연아동에 입학 격려 지원금 전달",롯데하이마트(대표 황영근)가 24일 입학을 기다리고 있는 조손가정 결연아동 33명을...,EC_M05_246826,"롯데하이마트, 해외 에너지 취약계층 아동, 친환경 태양광 랜턴 솔라미 전달",EC,0,롯데하이마트는 해외 에너지 소외계층 아동들을 위해 친환경 태양광 랜턴 솔라미를 전달...,"롯데하이마트, 해외 결연 아동에 태양광 랜턴 기증"
...,...,...,...,...,...,...,...,...,...
131995,PO_M08_394777,"\""서울시장 가상 양자대결…오세훈 49.7% 송영길 36.9%\""",6·1 지방선거에서 국민의힘 서울시장 후보로 나서는 오세훈 현 서울시장이 더불어민주...,PO_M08_386027,"\""국민힘 김은혜· 김동연, 근소한 열세\""",PO,0,국민의힘 김은혜 경기지사 후보가 더불어민주당 김동연 경기지사 후보를 오차밤위 밖에서...,"김은혜 47.9% vs 김동연 39.4%… 경기도민, 국정안정론에 51.3% 응답"
131996,SO_M07_155613,울산지검 '공무상 재해보상제도 안내서' 발간해 눈길,울산지방검찰청은 공무원 재해보상법 시행 후 처음으로 「공무상 재해보상제도 안내서」를...,SO_M02_126795,"전북, 중대재해 예방·체계 대응 계획 마련",SO,0,전북도는 오는 27일 '중대재해처벌 등에 관한 법률' 시행에 따라 중대재해 사전예방...,중대재해법 내일부터 시행...전북도 대응계획 마련
131997,GB_M11_060449,7m 악어 배 속에서 나온 30대 청년의 두 다리…주민들 '통곡',인도네시아령 파푸아의 한 마을에서 악어의 공격을 받고 실종된 30대 청년의 유해가 ...,GB_M11_055618,"中, \""한국서 참전군 유해 송환\""",GB,0,중국이 2일 한국으로부터 한국전쟁 참전군인 유해를 돌려받고 '영웅의 귀환'이라며 대...,"\""영웅 돌아왔다\""…한국전 전몰자 유해송환 中 애국주의 강조"
131998,EC_M05_237797,"하나금융그룹, '하나 소셜벤처 아카데미' 3기 모집",하나금융그룹은 창업 교육을 통해 사회혁신 창업가를 육성하는 프로그램 '하나 소셜벤처...,EC_M04_015469,‘하나 소셜벤처 아카데미' 4개 대학 신규 선정,EC,0,중소기업청은 청년 창업 활성화와 창업 붐 확산을 위해 우수한 창업 인프라와 역량을 ...,중기청 창업선도대학 6곳 신규선정
